In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import os, cv2, random
# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list the files in the input directory
from subprocess import check_output
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import MinMaxScaler 
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import LabelEncoder
from sklearn.grid_search import GridSearchCV
from PIL import Image
import os
from subprocess import check_output
print(check_output(["ls", "../input"]).decode("utf8"))

# Any results you write to the current directory are saved as output.

In [2]:
#读取图像
def getData():
    TRAIN_DIR = '../input/train/'
    TEST_DIR = '../input/test/'
    train_dogs =   [(TRAIN_DIR+'dog.'+str(num)+'.jpg', 1) for num in range(9375,12500)]
    train_cats =   [(TRAIN_DIR+'cat.'+str(num)+'.jpg', 0) for num in range(9375,12500)]
    #取测试集，这个地方需要提前占位，-1就是占位符。
    test_images =  [(TEST_DIR+str(num)+'.jpg', -1) for num in range(1,12501)]
    train_images = train_dogs+ train_cats
    random.shuffle(train_images)
    return train_images,test_images
train_images,test_images = getData()
#print(train_images)

In [3]:
#将图像转成dataframe的数组
def imgToDataFrame(images):
    df1 = pd.DataFrame(columns = [i for i in range(3*256)])
    df2 = pd.DataFrame(columns = [0])
    listx = []
    listy = []
    #i=1
    #print(len(images))
    for img in images:
        aimg = Image.open(img[0])
        aimg=aimg.resize((64,64),Image.ANTIALIAS)
        
        pix_val = list(aimg.getdata())
        pix_val_flat = [x for sets in pix_val for x in sets]
        #print(pix_val_flat)
        pix_val_flat = aimg.histogram()
        listx.append(pix_val_flat)
        listy.append(img[1])
        #df1=df1.append(pd.DataFrame(np.array([pix_val_flat]),columns = [i for i in range(3*64*64)]))
        #df2=df2.append(pd.DataFrame(np.array([img[1]]),columns = [0]))
        #if i>=100:break
        #i+=1
    df1 = pd.DataFrame(listx,columns = [i for i in range(256*3)])
    df2 = pd.DataFrame(listy,columns = [0])
    return df1,df2
xtrain,ytrain = imgToDataFrame(train_images)
xtest,_ = imgToDataFrame(test_images)
xtrain.head()

In [4]:
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC, LinearSVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import GaussianNB
#from sklearn.cross_validation import cross_val_score  
from sklearn.decomposition import PCA

In [15]:
#随机森林
#forest = RandomForestClassifier(n_estimators=50)
#param_grid = { 
 #   'n_estimators': [100,200,300,400,500],  'max_features': ['auto', 'log2']
#}
#CV_forest= GridSearchCV(estimator=forest, param_grid=param_grid, cv=5,scoring='neg_log_loss')
#CV_forest.fit(xtrain,ytrain)
#Ytest = CV_forest.predict_proba(xtest)
#forest.fit(xtrain,ytrain)
#Ytest = forest.predict_proba(xtest)
#print(Ytest)
ytrain[0]

In [16]:
params = {'C':[1, 10, 50, 100, 500, 1000], 'tol': [0.001, 0.0001, 0.005]}
logic = LogisticRegression()
CV_logic= GridSearchCV(estimator=logic, param_grid=params, cv=5,scoring='neg_log_loss')
print(len(xtrain))
print(ytrain.values)
CV_logic.fit(xtrain,ytrain[0])
Ytest = CV_logic.predict_proba(xtest)
#logic.fit(xtrain,ytrain)
#Ytest = logic.predict_proba(xtest)
print(Ytest)

In [17]:
#组合成结果
result = pd.DataFrame(Ytest,columns = ['label','label1'])
result.insert(0,"id",[i for i in range(1,len(Ytest)+1)])
result = result.drop('label1',axis = 1)
result.head()
result.to_csv('dogvscatsamples.csv', index=False)